In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
from collections import OrderedDict
from matplotlib.backends.backend_pdf import PdfPages

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from adapters import *
from dataset import NumpyCsvDataSet
from runner import ExptRunner
from networks.autoencoder import AutoEncoder
from networks.composedautoencoder import ComposedAutoEncoder
from networks.DenseAutoEncoder import DenseAutoEncoder
from networks.ConvVae import ConvVae
from networks.vae import VAE
from networks.etpr.autoencoder import autoencoder as etprautoenc
from networks.dense import Dense
from networks.dense import DenseForPolicy
from networks.imageencoder import ImageEncoder
from networks.multinet import *
from networks.lossfunctions import *
from helpers import ReportResults
from policyeval import PolicyEvaluator
from policyeval import eval_policy_accuracy
import utils

In [3]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

In [4]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [5]:
start = time.time()
data_folder = 'data/demoplanner/version1'
# v1_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v1_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 2.314089059829712 sec


In [6]:
start = time.time()
data_folder = 'data/demoplanner/version2'
# v2_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v2_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 11.10836410522461 sec


In [7]:
test_data = v2_test_data.data
# train_data = v2_train_data.data

In [8]:
# print (train_data.shape)
# print (train_data.device)

print (test_data.shape)
print (test_data.device)

torch.Size([6648, 2058])
cpu


In [9]:
t_cpu = test_data.data.cpu()
start_indices = []
index = 0
while index < len(t_cpu):
    start_indices.append(index)
    start, goal = t_cpu[index, :2], t_cpu[index, 2:4]
    index += int(max(np.abs(goal - start)))
print (start_indices)
print (len(start_indices))

[0, 22, 31, 34, 43, 50, 70, 86, 90, 106, 114, 122, 138, 152, 157, 173, 184, 191, 211, 219, 224, 226, 249, 260, 269, 279, 290, 307, 324, 340, 355, 367, 376, 400, 412, 426, 436, 453, 474, 485, 489, 509, 530, 536, 545, 554, 579, 597, 608, 621, 643, 651, 656, 664, 676, 683, 692, 706, 713, 740, 759, 774, 792, 802, 819, 841, 853, 878, 898, 905, 914, 925, 934, 944, 954, 974, 984, 998, 1007, 1008, 1015, 1025, 1043, 1061, 1065, 1089, 1094, 1103, 1114, 1126, 1129, 1152, 1160, 1177, 1183, 1196, 1207, 1232, 1243, 1265, 1274, 1293, 1302, 1316, 1328, 1350, 1358, 1371, 1375, 1397, 1402, 1422, 1439, 1442, 1454, 1461, 1481, 1500, 1516, 1536, 1550, 1559, 1565, 1577, 1601, 1626, 1645, 1669, 1681, 1694, 1702, 1720, 1729, 1738, 1753, 1777, 1792, 1793, 1794, 1802, 1814, 1818, 1836, 1842, 1865, 1887, 1909, 1915, 1925, 1931, 1941, 1963, 1979, 1998, 2023, 2042, 2068, 2093, 2096, 2106, 2124, 2136, 2149, 2160, 2180, 2194, 2206, 2213, 2236, 2242, 2248, 2260, 2269, 2291, 2304, 2312, 2322, 2336, 2350, 2360, 2380, 2

In [ ]:
def get_trajectory(start_indices, traj_index):
    rstart = start_indices[traj_index]
    rend = start_indices[traj_index+1] if traj_index < len(start_indices)-1 else len(t_cpu)
    return t_cpu[rstart:rend, :]
    
def get_trajectories(start_indices):
    for i in range(len(start_indices)):
        yield i, get_trajectory(start_indices, i)

In [ ]:
pretrain_policy = Dense([4, 16, 16, 6], use_last_act=False, prefix='enc')
pretrain_policy.load_state_dict(torch.load('runs/pretrained/04-08-10-24-50-Policy-V2-Dense-CE/autoenc.pth', 
                                  map_location=device))

pretrain_dynamics = Dense([4, 16, 2], prefix='enc')
pretrain_dynamics.load_state_dict(torch.load('runs/pretrained/03-31-18-02-52-Dynamics-V1-Dense-SmoothL1/autoenc.pth', 
                                    map_location=device))

pretrain_imgEncDec = ComposedAutoEncoder(layers_channels=[16,16,16,16], useMaxPool=True, device=device).to(device)
pretrain_imgEncDec.load_state_dict(torch.load('runs/pretrained/03-28-14-10-20-DemoPl-V2-ComposedAutoEncoderWithMaxPool-SmoothL1-/autoenc.pth',
                               map_location=device))

pretrain_envEncDec_chkpt = torch.load(
    'runs/pretrained/04-20-11-03-59-EnvAutoEnc-V2-DenseAutoEncoder-MSE/train_checkpoint.tar',map_location=device)
pretrain_envEncDec = pretrain_envEncDec_chkpt['model']

pretrain_ImgToEnvBaseLine_chkpt = torch.load(
    'runs/pretrained/04-21-12-51-48-ImageToEnvFunctionBaseLine-V2-3Conv2Dense-MSE/train_checkpoint.tar',map_location=device)
pretrain_ImgToEnvBaseLine = pretrain_ImgToEnvBaseLine_chkpt['model']

In [39]:
adjusted_ImgToEnv_BaseLine = nn.Sequential(OrderedDict([
    ('Conv', ImageEncoder(1, [16,16,16], 'imgenc', useMaxPool=True, addFlatten=True)),
    ('Dense', Dense([256, 4]))
]))
adjusted_ImgToEnv_BaseLine.load_state_dict(pretrain_ImgToEnvBaseLine.state_dict())

<All keys matched successfully>

In [62]:
pretrain_dynamics

Dense(
  (net): Sequential(
    (enc_fc0): Linear(in_features=4, out_features=16, bias=True)
    (enc_relu0): ReLU()
    (enc_fc1): Linear(in_features=16, out_features=2, bias=True)
    (enc_sig1): Sigmoid()
  )
)

In [ ]:
log_folder = 'runs/05-05/run1/05-05-10-07-01-FineTunePreTrainedEndToEnd-V2-EndToEndEnv-L1'
chkpt = torch.load(log_folder + '/train_checkpoint.tar', map_location=device)
net = chkpt['model']
label_adapter = lambda data: torch.cat((data[:, 2:4], data[:, 1028:1032], data[:, 1034:]), dim=1)
e = PolicyEvaluator(net, distance_func=F.l1_loss, label_adapter=label_adapter)

In [70]:
print ('L1 Loss on Y: {}'.format(F.l1_loss(predictions[:, 0:2], labels[:, 0:2])))
print ('Policy Accuracy: {}'.format(eval_policy_accuracy(predictions[:, 2:4], labels[:, 2:4])))
print ('L1 Loss on dynamics: {}'.format(F.l1_loss(predictions[:, 4:6], labels[:, 4:6])))

L1 Loss on Y: 1.126419186592102
Policy Accuracy: 58.82352828979492
L1 Loss on dynamics: 6.183617115020752


In [110]:
errors = torch.zeros((len(start_indices), 5))
for index, trajectory in get_trajectories(start_indices):
    goal_err, step_err, labels, predictions = e.eval_single_trajectory(trajectory)
    errors[index][0] = index
    errors[index][1] = len(trajectory)
    errors[index][2] = F.l1_loss(predictions[:, 0:2], labels[:, 0:2])
    errors[index][3] = eval_policy_accuracy(predictions[:, 2:4], labels[:, 2:4])
    errors[index][4] = F.l1_loss(predictions[:, 4:6], labels[:, 4:6])

In [111]:
np.savetxt(log_folder + '/trajectory_eval_metrics.csv', errors.numpy(), delimiter=',')

In [115]:
indices = [torch.argmin(errors[:, 2]).item(), torch.argmax(errors[:, 2]).item(),
           torch.argmin(errors[:, 3]).item(), torch.argmax(errors[:, 3]).item()]
print (indices)
print (errors[indices])

[183, 486, 499, 368]
tensor([[183.0000,   2.0000,   1.5250,   0.0000,   2.7497],
        [486.0000,  23.0000,  18.8735,   4.3478,  43.9181],
        [499.0000,  13.0000,   6.5673,   0.0000,  15.5889],
        [368.0000,  11.0000,   3.2069,  72.7273,   7.8180]])


In [89]:
traj = get_trajectory(start_indices, index)

In [90]:
goal_err, step_err, labels, predictions = e.eval_single_trajectory(traj)
print (len(traj))
print (goal_err)
print (step_err)

11
tensor(15.5210)
tensor(15.4023)


In [91]:
print (torch.cat((labels[:, :2], predictions[:, :2]), dim=1))
print (torch.cat((labels[:, 2:6], predictions[:, 2:6]), dim=1))

tensor([[1.7000e+01, 2.7000e+01, 6.1141e-08, 2.6649e-08],
        [1.7000e+01, 2.7000e+01, 1.4571e-06, 1.3033e-09],
        [1.7000e+01, 2.7000e+01, 1.4571e-06, 1.3033e-09],
        [1.7000e+01, 2.7000e+01, 1.4571e-06, 1.3033e-09],
        [1.7000e+01, 2.7000e+01, 1.4571e-06, 1.3033e-09],
        [1.7000e+01, 2.7000e+01, 1.4571e-06, 1.3033e-09],
        [1.7000e+01, 2.7000e+01, 1.4571e-06, 1.3033e-09],
        [1.7000e+01, 2.7000e+01, 1.4571e-06, 1.3033e-09],
        [1.7000e+01, 2.7000e+01, 1.4571e-06, 1.3033e-09],
        [1.7000e+01, 2.7000e+01, 1.4571e-06, 1.3033e-09],
        [1.7000e+01, 2.7000e+01, 1.4571e-06, 1.3033e-09]])
tensor([[ 1.0000,  0.0000,  7.0000, 24.0000, -1.0000, -1.0000,  8.0108, 22.3381],
        [ 1.0000,  0.0000,  8.0000, 24.0000, -1.0000, -1.0000,  8.0119, 22.3391],
        [ 1.0000,  0.0000,  9.0000, 24.0000, -1.0000, -1.0000,  8.0130, 22.3402],
        [ 1.0000,  0.0000, 10.0000, 24.0000, -1.0000, -1.0000,  8.0140, 22.3413],
        [ 1.0000,  0.0000, 11.000

In [ ]:
for index in indices:
    print ('-------------------------------------------------------------')
    print ('                         {}'.format(index))
    print ('-------------------------------------------------------------')
    with PdfPages(log_folder + '/traj-' + str(index) + '.pdf') as pdf:
        for i in range(len(traj)):
            fig = plt.figure()
            plt.subplot(1,2,1)
            plt.imshow(labels[i, 6:].reshape(32,32), cmap=plt.get_cmap("gray"))
            plt.subplot(1,2,2)
            plt.imshow(predictions[i, 6:].reshape(32,32), cmap=plt.get_cmap("gray"))
            pdf.savefig(fig)
            plt.show()

In [27]:
print ('Trajectory with highest goal error = {}'.format(torch.argmax(errors[:, 1])))
print ('Trajectory with highest average step error = {}'.format(torch.argmax(errors[:, 2])))
print ('Trajectory Length, Goal Error, Average Error')
for i in range(len(errors)):
    print('{},{},{}'.format(*errors[i]))

Trajectory with highest goal error = 386
Trajectory with highest average step error = 386
Trajectory Length, Goal Error, Average Error
22.0,12.960399627685547,4.884669780731201
9.0,4.852686882019043,2.695817708969116
3.0,1.595874309539795,1.0638898611068726
9.0,5.459317684173584,2.786447286605835
7.0,6.841001033782959,3.9084365367889404
20.0,14.134435653686523,6.362017631530762
16.0,13.942781448364258,6.975234508514404
4.0,2.437255382537842,1.3359044790267944
16.0,9.144655227661133,4.250625133514404
8.0,5.237362861633301,2.5759902000427246
8.0,6.318533897399902,3.088604688644409
16.0,9.571662902832031,3.9311604499816895
14.0,10.939911842346191,4.983663082122803
5.0,4.435301780700684,2.461456537246704
16.0,9.867547988891602,4.4905571937561035
11.0,10.018267631530762,5.24059534072876
7.0,4.5808610916137695,2.244675636291504
20.0,16.234203338623047,7.611097812652588
8.0,4.736490249633789,2.309919834136963
5.0,4.173446178436279,2.204035520553589
2.0,1.3381853103637695,0.913787841796875
23.

In [23]:
traj_no = 386 
rstart = start_indices[traj_no]
rend = start_indices[traj_no+1] if traj_no < len(start_indices) else len(test_data)
trajectory = t_cpu[rstart:rend, :]
f = torch.cat((trajectory[:, :4], trajectory[:, 1028:1030]), dim=1)
print (len(f))
print (f)

25
tensor([[28., 27.,  3.,  4., -1.,  0.],
        [27., 27.,  3.,  4., -1.,  0.],
        [26., 27.,  3.,  4., -1., -1.],
        [25., 26.,  3.,  4., -1., -1.],
        [24., 25.,  3.,  4., -1., -1.],
        [23., 24.,  3.,  4., -1., -1.],
        [22., 23.,  3.,  4., -1., -1.],
        [21., 22.,  3.,  4., -1., -1.],
        [20., 21.,  3.,  4., -1., -1.],
        [19., 20.,  3.,  4., -1., -1.],
        [18., 19.,  3.,  4., -1., -1.],
        [17., 18.,  3.,  4., -1., -1.],
        [16., 17.,  3.,  4., -1., -1.],
        [15., 16.,  3.,  4., -1., -1.],
        [14., 15.,  3.,  4., -1., -1.],
        [13., 14.,  3.,  4., -1., -1.],
        [12., 13.,  3.,  4., -1., -1.],
        [11., 12.,  3.,  4., -1., -1.],
        [10., 11.,  3.,  4., -1., -1.],
        [ 9., 10.,  3.,  4., -1., -1.],
        [ 8.,  9.,  3.,  4., -1., -1.],
        [ 7.,  8.,  3.,  4., -1., -1.],
        [ 6.,  7.,  3.,  4., -1., -1.],
        [ 5.,  6.,  3.,  4., -1., -1.],
        [ 4.,  5.,  3.,  4., -1., -1.

In [24]:
goal_err, step_err, pred_traj, orig_traj, policy_pred, dyn_pred = e.eval_single_trajectory(trajectory)

print ('Last step error = {}'.format(goal_err))
print ('Average error over all steps = {}'.format(step_err))
print ('Original Trajectory and Predicted Trajectory:')
print (torch.cat((orig_traj, pred_traj), dim=1))
print ('Stepwise deviation from original trajectory:')
print (torch.abs(orig_traj-pred_traj))
print ('Prediction from Policy:')
print (policy_pred - 1.0) # Using minus to bring the policy output to the values we expect
print ('Prediction from Dynamics:')
print (dyn_pred * 2.0 - 1.0)  # Again bringing the dynamics prediction to the expected range

Last step error = 21.83930206298828
Average error over all steps = 10.87393569946289
Original Trajectory and Predicted Trajectory:
tensor([[27.0000, 27.0000, 27.8958, 26.9159],
        [26.0000, 27.0000, 27.7920, 26.8327],
        [25.0000, 26.0000, 27.6888, 26.7504],
        [24.0000, 25.0000, 27.5860, 26.6689],
        [23.0000, 24.0000, 27.4837, 26.5883],
        [22.0000, 23.0000, 27.3819, 26.5085],
        [21.0000, 22.0000, 27.2805, 26.4296],
        [20.0000, 21.0000, 27.1796, 26.3515],
        [19.0000, 20.0000, 27.0792, 26.2743],
        [18.0000, 19.0000, 26.9792, 26.1978],
        [17.0000, 18.0000, 26.8797, 26.1222],
        [16.0000, 17.0000, 26.7806, 26.0474],
        [15.0000, 16.0000, 26.6819, 25.9734],
        [14.0000, 15.0000, 26.5837, 25.9002],
        [13.0000, 14.0000, 26.4860, 25.8278],
        [12.0000, 13.0000, 26.3887, 25.7562],
        [11.0000, 12.0000, 26.2918, 25.6854],
        [10.0000, 11.0000, 26.1953, 25.6154],
        [ 9.0000, 10.0000, 26.0993, 25.54

In [ ]:
np.set_printoptions(**opt)